In [1]:
from pyspark.sql import SparkSession
import os
# from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

In [2]:
schema = "`platformGameId` STRING, `matchId` STRING, `esportsGameId` STRING, `tournamentId` STRING, `teamMapping`" \
"MAP<STRING, STRING>, `participantMapping` MAP<STRING, STRING>"
vct_intl_mapping_data = spark.read.schema(schema) \
                                .json("./mappings/mapping_data_intl.json", multiLine=True)

vct_intl_mapping_data.printSchema()
# vct_intl_mapping_data.show()

root
 |-- platformGameId: string (nullable = true)
 |-- matchId: string (nullable = true)
 |-- esportsGameId: string (nullable = true)
 |-- tournamentId: string (nullable = true)
 |-- teamMapping: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- participantMapping: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [3]:
vct_champions_2024_tournament_id = "112053452805788444"
vct_champions_2022_tournament_id = "108871629797692793"

In [4]:
from pyspark.sql.functions import size, col
vct_champions_2022_game_mapping = vct_intl_mapping_data.filter(vct_intl_mapping_data.tournamentId == vct_champions_2022_tournament_id)
vct_champions_2024_game_mapping = vct_intl_mapping_data.filter(vct_intl_mapping_data.tournamentId == vct_champions_2024_tournament_id)
                                                              # & (size(vct_intl_mapping_data["teamMapping"]) == 2))
vct_champions_2024_game_mapping.show()

+--------------------+------------------+------------------+------------------+--------------------+--------------------+
|      platformGameId|           matchId|     esportsGameId|      tournamentId|         teamMapping|  participantMapping|
+--------------------+------------------+------------------+------------------+--------------------+--------------------+
|val:bd829eaf-f7b8...|112806895244190011|112806895244190013|112053452805788444|{20 -> 1057517659...|{1 -> 10611644096...|
|val:dbb490b7-9645...|112806895243272439|112806895243272440|112053452805788444|{20 -> 1056658698...|{1 -> 10979508616...|
|val:54bd8aed-23fe...|112806895243272435|112806895243272436|112053452805788444|{20 -> 1056653782...|{1 -> 10772377267...|
|val:ef851500-b356...|112806895244255567|112806895244255570|112053452805788444|{20 -> 1056658698...|{1 -> 10665214873...|
|val:5c96bbf8-d78a...|112806895244452189|112806895244452190|112053452805788444|{19 -> 1086961516...|{1 -> 10772377267...|
|val:ee7ff50a-7cbd...|11

In [5]:
from pyspark.sql.functions import *
vct_champions_2024_game_mapping.select("teamMapping").show(truncate=False)

+----------------------------------------------------+
|teamMapping                                         |
+----------------------------------------------------+
|{20 -> 105751765959424656, 21 -> 105788004225594471}|
|{20 -> 105665869861005803, 21 -> 105680908693496715}|
|{20 -> 105665378258561492, 21 -> 105751765959424656}|
|{20 -> 105665869861005803, 21 -> 105680972836508184}|
|{19 -> 108696151689107350, 20 -> 105751765959424656}|
|{20 -> 107795656627193642, 21 -> 105751765959424656}|
|{20 -> 108696151689107350, 21 -> 105795137629057449}|
|{20 -> 108696151689107350, 21 -> 105665869861005803}|
|{20 -> 105788004225594471, 21 -> 105751765959424656}|
|{20 -> 108696151689107350, 21 -> 105751765959424656}|
|{20 -> 106013381894847385, 21 -> 105788004225594471}|
|{20 -> 107795656627193642, 21 -> 105751765959424656}|
|{20 -> 107795656627193642, 21 -> 105751765959424656}|
|{20 -> 105720659034682989, 21 -> 105795137629057449}|
|{20 -> 110773882819766782, 21 -> 105623681847525855}|
|{20 -> 11

In [6]:
vct_champions_2024_game_mapping.select("participantMapping").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|participantMapping                                                                                                                                                                                                                                         |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{1 -> 106116440965658227, 2 -> 107769214873453994, 3 -> 106489999216559638, 4 -> 107891889389481497, 5 -> 106156773024391652, 6 -> 107723772674746660, 7 -> 107764993804063401, 8 -> 107735298728316922, 9 -> 109630636704842016, 10 -> 10964

In [7]:
vct_champions_2024_game_mapping \
    .select("matchId") \
    .distinct() \
    .count() 

34

In [8]:
vct_champions_2024_game_mapping.write.json("./vct_champions_2024_game_mapping.json")
vct_champions_2022_game_mapping.write.json("./vct_champions_2022_game_mapping.json")